In [5]:
from sklearn.metrics import cohen_kappa_score
import csv
from collections import defaultdict
from itertools import islice

#PART 2

#Method done for the part 1 of the assignment 
#Receives the name of the file 
#Creates a list with the Cohen's kappa agreement score for pair of annotators, a list with the score for each individual annotator obtained with the average Cohen's kappa agreement score and a list with the content of the file
#This method is done using dictionaries that helps the user find the information of an annotator by looking for his name score_annotator["Mateo"]
#To look for the Cohen's kappa agreement score for pair of annotators in the dictionary, it will be needed to write the name of the annotaator with a "|" followed by the other annotator's name
def cohen_annotation (filename):
    csvfile=open(filename,encoding='Latin1')
    csvreader = csv.reader(csvfile) 
    header = []
    header = next(csvreader)
    rows = []
    rows.append(header)
    annotators = defaultdict(list)
    for row in csvreader:
        rows.append(row)
        for i in range(len(row)-1):
            annotators[header[i+1]].append(1 if row[i+1] == 'True' else 0)
    cohen_annotators = defaultdict(list)
    score_annotator = defaultdict(list)
    for i in range(4):
        for j in range(4):
            cohen_annotators[header[i+1]+'|' +header[j+1]].append(cohen_kappa_score(annotators[header[i+1]],annotators[header[j+1]]))
        return_values = []
        for k,v in cohen_annotators.items():
            if k.startswith(header[i+1]):
                return_values.append(sum(v))
        return_values.pop(i)
        score_annotator[header[i+1]].append(sum(return_values)/3)
    return cohen_annotators, score_annotator,rows

labels={"masking_and_distancing","vaccination","lockdowns"}

#Initialize the auxiliar lists
cohen_kappa_twitter= list()
mean_score_twitter= list()
dataset_twitter=list()
cohen_kappa_nyt=list()
mean_score_nyt=list()
dataset_nyt=list()

#For each topic there is a file, so this runs automatically through each file
for name in labels:
    for i in range(4):
        filename_twitter="twitter_topic_"+str(i)+"_"+name+".csv"
        a,b,c=cohen_annotation(filename_twitter)
        cohen_kappa_twitter.append(a)
        mean_score_twitter.append(b)
        dataset_twitter.append(c)
    for i in range(3):
        filename_nyt="nyt_topic_"+str(i)+"_"+name+".csv"
        d,e,f=cohen_annotation(filename_nyt)
        cohen_kappa_nyt.append(d)
        mean_score_nyt.append(e)
        dataset_nyt.append(f)

        
#PART 2

#Method done for the part 2 of the assignment 
#Receives the name of the file 
#Creates a list with the content of the file manipulated
#This method eliminates any labels for annotators whose average kappa score is less than 0.2 and assigns a final label to each text 
#For the assignment of the label, the method gives the label that is most frequent between the remaining annotations
#In case of a tie, it gives the label that has higher-reliability, this means, the higher kappa scores on average
def del_annotators(filename):
    cohen, average_score,dataset=cohen_annotation(filename)
    new_dataset=list()
    j=0
    new_average_score=average_score.copy()
    for x in average_score.values():
        if(sum(x)<0.200):
            for column in dataset:
                del column[j+1]
            del new_average_score[next(islice(new_average_score, j, None))] 
            j=j-1
        j=j+1
    aux=list()
    for column in dataset[1:]:
        trues=0
        falses=0
        aux.append(column[0])
        for label in column[1:]:
            if(label=='True'):
                trues=trues+1
            elif(label=='False'):
                falses=falses+1
        if(trues>falses):
            aux.append("True")
        elif(falses>trues):
            aux.append("False")
        elif(trues==falses):
            max=0
            avgTrue=0
            avgFalse=0
            numTrue=1
            numFalse=1
            for annotation,label in zip(dataset[0][1:],column[1:]):
                if(sum(new_average_score[annotation])>max):
                    if(label=='True'):
                        avgTrue=(avgTrue+sum(new_average_score[annotation]))/numTrue
                        numTrue=numTrue+1
                    elif(label=='False'):
                        avgFalse=(avgFalse+sum(new_average_score[annotation]))/numFalse
                        numFalse=numFalse+1
            aux.append("True" if avgTrue>avgFalse else "False")
    return aux

#For each topic there is a file, so this runs automatically through each file
for name in labels:
    new_dataset_twitter=list()
    new_dataset_nyt=list()
    for i in range(4):
        filename_twitter="twitter_topic_"+str(i)+"_"+name+".csv"
        n=del_annotators(filename_twitter)
        new_dataset_twitter.extend(n)
    with open('twitter_topic_' + name + ".csv", 'w', encoding='Latin1', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Text","Label"])
        for i in range(0,len(new_dataset_twitter),2):
            writer.writerow((new_dataset_twitter[i],new_dataset_twitter[i+1]))          
    f.close()
    for i in range(3):
        filename_nyt="nyt_topic_"+str(i)+"_"+name+".csv"
        m=del_annotators(filename_nyt)
        new_dataset_nyt.extend(m)
    with open('nyt_topic_' + name + ".csv", 'w',encoding='Latin1', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Text","Label"])
        for i in range(0,len(new_dataset_nyt),2):
            writer.writerow([new_dataset_nyt[i],new_dataset_nyt[i+1]])  
    f.close()